<a href="https://colab.research.google.com/github/nitinsaimajji/final-year/blob/main/sms_DL_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
!pip install scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

In [111]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam

In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

# Load your training dataset
train_df = pd.read_csv('new_sms_train.csv')
# Load your testing dataset
test_df = pd.read_csv('new_sms_test.csv')


In [113]:
# Assuming test_df contains a 'label' column indicating the class (e.g., 'spam' or 'not spam')

# Separate samples by class
spam_samples_test = test_df[test_df['label'] == 'spam']
not_spam_samples_test = test_df[test_df['label'] == 'not_spam']

# Determine the number of samples to select from each class
num_samples_per_class_test = min(len(spam_samples_test), len(not_spam_samples_test))
max_total_samples = 500  # Maximum total number of samples

# Calculate the number of samples to select for test set, ensuring it doesn't exceed the maximum
num_samples_test = min(num_samples_per_class_test * 2, max_total_samples)

# Sample an equal number of instances from each class
sampled_spam_test = spam_samples_test.sample(n=num_samples_test // 2, random_state=42)
sampled_not_spam_test = not_spam_samples_test.sample(n=num_samples_test // 2, random_state=42)

# Concatenate the sampled dataframes
sampled_test_df = pd.concat([sampled_spam_test, sampled_not_spam_test])

# Shuffle the dataframe to mix spam and not spam samples
sampled_test_df = sampled_test_df.sample(frac=1, random_state=42)

print(len(sampled_test_df))


500


In [114]:
test_df=sampled_test_df

In [115]:
test_df.head()

,text,label
851,"Hi people, this is probably not the right subr...",not_spam
290,OMG! Have you heard about the new amazing weig...,spam
727,[We provide an advanced dataset on the Shopify...,not_spam
357,HAF U EVR WANTED TOO MAK MONY FAST? WLL SRCH N...,spam
332,OMG GUYS! You won't BELIEVE what I just found ...,spam


In [116]:
# Assuming train_df contains a 'label' column indicating the class (e.g., 'spam' or 'not spam')

# Separate samples by class
spam_samples_train = train_df[train_df['label'] == 'spam']
not_spam_samples_train = train_df[train_df['label'] == 'not_spam']

# Determine the number of samples to select from each class
num_samples_per_class_train = min(len(spam_samples_train), len(not_spam_samples_train))
max_total_samples_train = 800  # Maximum total number of samples

# Calculate the number of samples to select for the training set, ensuring it doesn't exceed the maximum
num_samples_train = min(num_samples_per_class_train * 2, max_total_samples_train)

# Sample an equal number of instances from each class
sampled_spam_train = spam_samples_train.sample(n=num_samples_train // 2, random_state=42)
sampled_not_spam_train = not_spam_samples_train.sample(n=num_samples_train // 2, random_state=42)

# Concatenate the sampled dataframes
sampled_train_df = pd.concat([sampled_spam_train, sampled_not_spam_train])

# Shuffle the dataframe to mix spam and not spam samples
sampled_train_df = sampled_train_df.sample(frac=1, random_state=42)

print(len(sampled_train_df))


800


In [117]:
train_df=sampled_train_df
print(len(train_df))

800


In [118]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [119]:
# Data Preprocessing
# Find the maximum number of unique words in the dataset
max_words = 100

# Find the maximum length of messages in the dataset
max_len = 50


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

# Preprocessing for training data
train_df['text'] = train_df['text'].apply(preprocess_text)

# Split the training dataset
X_train, y_train = train_df['text'], train_df['label']

# Tokenization
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Sequence and Padding
X_train_sequences = tokenizer.texts_to_sequences(X_train)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')

# Preprocessing for testing data
test_df['text'] = test_df['text'].apply(preprocess_text)

# Split the testing dataset
X_test, y_test = test_df['text'], test_df['label']

# Tokenization
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')



## Building and Training Models

## DNN

In [120]:
# dnn_model_modified = Sequential([
#     Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
#     Flatten(),
#     Dense(32, activation='sigmoid'),
#     Dropout(0.2),
#     Dense(16, activation='relu'),
#     Dropout(0.7),  # Increased dropout rate
#     Dense(1, activation='sigmoid')
# ])

def create_dnn_model(optimizer='adam',learning_rate=0.001):
    dnn_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Flatten(),
        Dense(32, activation='sigmoid'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.7),  # Increased dropout rate
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    dnn_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

# Wrap the Keras model in a scikit-learn compatible classifier
dnn_classifier = KerasClassifier(build_fn=create_dnn_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_dnn = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_dnn = GridSearchCV(estimator=dnn_classifier, param_grid=param_grid_dnn, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_dnn = grid_search_dnn.fit(X_train_padded, y_train)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


17/17 - 1s - loss: 0.7205 - accuracy: 0.5178 - 680ms/epoch - 40ms/step
Epoch 2/30
17/17 - 0s - loss: 0.7060 - accuracy: 0.5197 - 29ms/epoch - 2ms/step
Epoch 3/30
17/17 - 0s - loss: 0.7231 - accuracy: 0.4784 - 26ms/epoch - 2ms/step
Epoch 4/30
17/17 - 0s - loss: 0.6821 - accuracy: 0.5647 - 25ms/epoch - 1ms/step
Epoch 5/30
17/17 - 0s - loss: 0.7008 - accuracy: 0.5084 - 26ms/epoch - 2ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6980 - accuracy: 0.4972 - 27ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6652 - accuracy: 0.5966 - 26ms/epoch - 2ms/step
Epoch 8/30
17/17 - 0s - loss: 0.6493 - accuracy: 0.6154 - 27ms/epoch - 2ms/step
Epoch 9/30
17/17 - 0s - loss: 0.6114 - accuracy: 0.6998 - 29ms/epoch - 2ms/step
Epoch 10/30
17/17 - 0s - loss: 0.5461 - accuracy: 0.7392 - 26ms/epoch - 2ms/step
Epoch 11/30
17/17 - 0s - loss: 0.4924 - accuracy: 0.7842 - 27ms/epoch - 2ms/step
Epoch 12/30
17/17 - 0s - loss: 0.4081 - accuracy: 0.8574 - 26ms/epoch - 2ms/step
Epoch 13/30
17/17 - 0s - loss: 0.3719 - accura

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


17/17 - 1s - loss: 0.8146 - accuracy: 0.5178 - 776ms/epoch - 46ms/step
Epoch 2/30
17/17 - 0s - loss: 0.7239 - accuracy: 0.5197 - 25ms/epoch - 1ms/step
Epoch 3/30
17/17 - 0s - loss: 0.7052 - accuracy: 0.5310 - 25ms/epoch - 1ms/step
Epoch 4/30
17/17 - 0s - loss: 0.6913 - accuracy: 0.5553 - 25ms/epoch - 1ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6805 - accuracy: 0.5441 - 26ms/epoch - 2ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6979 - accuracy: 0.5047 - 32ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6731 - accuracy: 0.5516 - 40ms/epoch - 2ms/step
Epoch 8/30
17/17 - 0s - loss: 0.6560 - accuracy: 0.6023 - 28ms/epoch - 2ms/step
Epoch 9/30
17/17 - 0s - loss: 0.6077 - accuracy: 0.6660 - 37ms/epoch - 2ms/step
Epoch 10/30
17/17 - 0s - loss: 0.5532 - accuracy: 0.7167 - 32ms/epoch - 2ms/step
Epoch 11/30
17/17 - 0s - loss: 0.4581 - accuracy: 0.7936 - 31ms/epoch - 2ms/step
Epoch 12/30
17/17 - 0s - loss: 0.4003 - accuracy: 0.8349 - 29ms/epoch - 2ms/step
Epoch 13/30
17/17 - 0s - loss: 0.3321 - accura

...

17/17 - 1s - loss: 0.7986 - accuracy: 0.5066 - 742ms/epoch - 44ms/step
Epoch 2/30
17/17 - 0s - loss: 0.7233 - accuracy: 0.5122 - 29ms/epoch - 2ms/step
Epoch 3/30
17/17 - 0s - loss: 0.7098 - accuracy: 0.5141 - 30ms/epoch - 2ms/step
Epoch 4/30
17/17 - 0s - loss: 0.7072 - accuracy: 0.5047 - 35ms/epoch - 2ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6929 - accuracy: 0.5441 - 28ms/epoch - 2ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6915 - accuracy: 0.5197 - 30ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6765 - accuracy: 0.5253 - 31ms/epoch - 2ms/step
Epoch 8/30
17/17 - 0s - loss: 0.6565 - accuracy: 0.6004 - 28ms/epoch - 2ms/step
Epoch 9/30
17/17 - 0s - loss: 0.6323 - accuracy: 0.6285 - 31ms/epoch - 2ms/step
Epoch 10/30
17/17 - 0s - loss: 0.5784 - accuracy: 0.7148 - 31ms/epoch - 2ms/step
Epoch 11/30
17/17 - 0s - loss: 0.5513 - accuracy: 0.7148 - 35ms/epoch - 2ms/step
Epoch 12/30
17/17 - 0s - loss: 0.4686 - accuracy: 0.7917 - 30ms/epoch - 2ms/step
Epoch 13/30
17/17 - 0s - loss: 0.4086 - accura

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


17/17 - 1s - loss: 0.9473 - accuracy: 0.5262 - 698ms/epoch - 41ms/step
Epoch 2/30
17/17 - 0s - loss: 0.6965 - accuracy: 0.5655 - 41ms/epoch - 2ms/step
Epoch 3/30
17/17 - 0s - loss: 0.6978 - accuracy: 0.5318 - 33ms/epoch - 2ms/step
Epoch 4/30
17/17 - 0s - loss: 0.6954 - accuracy: 0.5618 - 31ms/epoch - 2ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6712 - accuracy: 0.5899 - 29ms/epoch - 2ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6297 - accuracy: 0.6330 - 31ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6281 - accuracy: 0.6442 - 28ms/epoch - 2ms/step
Epoch 8/30
17/17 - 0s - loss: 0.5533 - accuracy: 0.7397 - 28ms/epoch - 2ms/step
Epoch 9/30
17/17 - 0s - loss: 0.4761 - accuracy: 0.7640 - 29ms/epoch - 2ms/step
Epoch 10/30
17/17 - 0s - loss: 0.4572 - accuracy: 0.7959 - 28ms/epoch - 2ms/step
Epoch 11/30
17/17 - 0s - loss: 0.3997 - accuracy: 0.8221 - 28ms/epoch - 2ms/step
Epoch 12/30
17/17 - 0s - loss: 0.3644 - accuracy: 0.8483 - 27ms/epoch - 2ms/step
Epoch 13/30
17/17 - 0s - loss: 0.3342 - accura

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


25/25 - 1s - loss: 0.7405 - accuracy: 0.5125 - 785ms/epoch - 31ms/step
Epoch 2/30
25/25 - 0s - loss: 0.7102 - accuracy: 0.5025 - 56ms/epoch - 2ms/step
Epoch 3/30
25/25 - 0s - loss: 0.6963 - accuracy: 0.5375 - 61ms/epoch - 2ms/step
Epoch 4/30
25/25 - 0s - loss: 0.6566 - accuracy: 0.6012 - 54ms/epoch - 2ms/step
Epoch 5/30
25/25 - 0s - loss: 0.6029 - accuracy: 0.6712 - 49ms/epoch - 2ms/step
Epoch 6/30
25/25 - 0s - loss: 0.5248 - accuracy: 0.7337 - 63ms/epoch - 3ms/step
Epoch 7/30
25/25 - 0s - loss: 0.4230 - accuracy: 0.8000 - 57ms/epoch - 2ms/step
Epoch 8/30
25/25 - 0s - loss: 0.3527 - accuracy: 0.8400 - 58ms/epoch - 2ms/step
Epoch 9/30
25/25 - 0s - loss: 0.2770 - accuracy: 0.8750 - 53ms/epoch - 2ms/step
Epoch 10/30
25/25 - 0s - loss: 0.2302 - accuracy: 0.8963 - 64ms/epoch - 3ms/step
Epoch 11/30
25/25 - 0s - loss: 0.2007 - accuracy: 0.9013 - 56ms/epoch - 2ms/step
Epoch 12/30
25/25 - 0s - loss: 0.1836 - accuracy: 0.9000 - 51ms/epoch - 2ms/step
Epoch 13/30
25/25 - 0s - loss: 0.1362 - accura

In [121]:
# Display the results in a table
results_df_dnn = pd.DataFrame(grid_result_dnn.cv_results_)
results_df_dnn[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.982512,2
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.981264,8
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.982512,2
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.982512,2
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.978762,14
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.976261,15
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.980016,11
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.982512,2
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.982512,2
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.983761,1


In [122]:
# Print the best parameters and corresponding accuracy
print("Best Parameters (DNN): ", grid_result_dnn.best_params_)
print("Best Accuracy (DNN): ", grid_result_dnn.best_score_)

Best Parameters (DNN):  {'learning_rate': 0.001, 'optimizer': 'Nadam'}
Best Accuracy (DNN):  0.9837609003782865


In [123]:
best_dnn_model = grid_result_dnn.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_dnn_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)

16/16 - 0s - 61ms/epoch - 4ms/step
Test Accuracy: 0.976


In [124]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='spam')
recall_spam = recall_score(y_test, y_pred, pos_label='spam')
f1_spam = f1_score(y_test, y_pred, pos_label='spam')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not_spam')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not_spam')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not_spam')

print("Precision (spam):", precision_spam)
print("Recall (spam):", recall_spam)
print("F1-score (spam):", f1_spam)

print("Precision (non-spam):", precision_non_spam)
print("Recall (non-spam):", recall_non_spam)
print("F1-score (non-spam):", f1_non_spam)


Precision (spam): 0.9877049180327869
Recall (spam): 0.964
F1-score (spam): 0.9757085020242914
Precision (non-spam): 0.96484375
Recall (non-spam): 0.988
F1-score (non-spam): 0.9762845849802372


## LSTM

In [125]:
# lstm_model = Sequential([
#     Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
#     LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
#     Dropout(0.5),
#     LSTM(64),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.3),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])


def create_lstm_model(optimizer='adam', learning_rate=0.001):
    lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
        LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
        Dropout(0.5),
        LSTM(64),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_lstm = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_lstm = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid_lstm, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_lstm = grid_search_lstm.fit(X_train_padded, y_train)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 4s - loss: 0.6932 - accuracy: 0.4916 - 4s/epoch - 220ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6933 - accuracy: 0.5216 - 609ms/epoch - 36ms/step
Epoch 3/30
17/17 - 1s - loss: 0.6785 - accuracy: 0.5647 - 596ms/epoch - 35ms/step
Epoch 4/30
17/17 - 1s - loss: 0.4440 - accuracy: 0.8837 - 592ms/epoch - 35ms/step
Epoch 5/30
17/17 - 1s - loss: 0.2155 - accuracy: 0.9568 - 588ms/epoch - 35ms/step
Epoch 6/30
17/17 - 1s - loss: 0.0927 - accuracy: 0.9812 - 618ms/epoch - 36ms/step
Epoch 7/30
17/17 - 1s - loss: 0.1193 - accuracy: 0.9681 - 975ms/epoch - 57ms/step
Epoch 8/30
17/17 - 1s - loss: 0.0707 - accuracy: 0.9831 - 1s/epoch - 59ms/step
Epoch 9/30
17/17 - 1s - loss: 0.0526 - accuracy: 0.9906 - 1s/epoch - 85ms/step
Epoch 10/30
17/17 - 1s - loss: 0.0372 - accuracy: 0.9944 - 1s/epoch - 65ms/step
Epoch 11/30
17/17 - 1s - loss: 0.0504 - accuracy: 0.9850 - 1s/epoch - 76ms/step
Epoch 12/30
17/17 - 1s - loss: 0.0813 - accuracy: 0.9775 - 944ms/epoch - 56ms/step
Epoch 13/30
17/17 - 1s - lo

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 5s - loss: 0.6939 - accuracy: 0.4991 - 5s/epoch - 299ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6911 - accuracy: 0.5272 - 669ms/epoch - 39ms/step
Epoch 3/30
17/17 - 1s - loss: 0.6850 - accuracy: 0.5516 - 681ms/epoch - 40ms/step
Epoch 4/30
17/17 - 1s - loss: 0.3983 - accuracy: 0.8874 - 661ms/epoch - 39ms/step
Epoch 5/30
17/17 - 1s - loss: 0.2322 - accuracy: 0.9568 - 699ms/epoch - 41ms/step
Epoch 6/30
17/17 - 1s - loss: 0.3645 - accuracy: 0.9137 - 673ms/epoch - 40ms/step
Epoch 7/30
17/17 - 1s - loss: 0.2491 - accuracy: 0.9325 - 692ms/epoch - 41ms/step
Epoch 8/30
17/17 - 1s - loss: 0.1802 - accuracy: 0.9512 - 675ms/epoch - 40ms/step
Epoch 9/30
17/17 - 1s - loss: 0.1113 - accuracy: 0.9812 - 682ms/epoch - 40ms/step
Epoch 10/30
17/17 - 1s - loss: 0.0561 - accuracy: 0.9869 - 675ms/epoch - 40ms/step
Epoch 11/30
17/17 - 1s - loss: 0.0575 - accuracy: 0.9869 - 829ms/epoch - 49ms/step
Epoch 12/30
17/17 - 1s - loss: 0.0658 - accuracy: 0.9869 - 880ms/epoch - 52ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
17/17 - 5s - loss: 0.6943 - accuracy: 0.4934 - 5s/epoch - 315ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6934 - accuracy: 0.5009 - 687ms/epoch - 40ms/step
Epoch 3/30
17/17 - 1s - loss: 0.6906 - accuracy: 0.5385 - 715ms/epoch - 42ms/step
Epoch 4/30
17/17 - 1s - loss: 0.6052 - accuracy: 0.6735 - 712ms/epoch - 42ms/step
Epoch 5/30
17/17 - 1s - loss: 0.2881 - accuracy: 0.9644 - 708ms/epoch - 42ms/step
Epoch 6/30
17/17 - 1s - loss: 0.2113 - accuracy: 0.9475 - 704ms/epoch - 41ms/step
Epoch 7/30
17/17 - 1s - loss: 0.1143 - accuracy: 0.9794 - 696ms/epoch - 41ms/step
Epoch 8/30
17/17 - 1s - loss: 0.0905 - accuracy: 0.9775 - 702ms/epoch - 41ms/step
Epoch 9/30
17/17 - 1s - loss: 0.0675 - accuracy: 0.9737 - 705ms/epoch - 41ms/step
Epoch 10/30
17/17 - 1s - loss: 0.0438 - accuracy: 0.9925 - 726ms/epoch - 43ms/step
Epoch 11/30
17/17 - 1s - loss: 0.0362 - accuracy: 0.9944 - 727ms/epoch - 43ms/step
Epoch 12/30
17/17 - 1s - loss: 0.0314 - accuracy: 0.9962 - 705ms/epoch - 41ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 4s - loss: 0.6934 - accuracy: 0.5112 - 4s/epoch - 222ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6933 - accuracy: 0.4794 - 677ms/epoch - 40ms/step
Epoch 3/30
17/17 - 1s - loss: 0.6929 - accuracy: 0.4925 - 666ms/epoch - 39ms/step
Epoch 4/30
17/17 - 1s - loss: 0.6901 - accuracy: 0.5243 - 690ms/epoch - 41ms/step
Epoch 5/30
17/17 - 1s - loss: 0.4545 - accuracy: 0.8596 - 691ms/epoch - 41ms/step
Epoch 6/30
17/17 - 1s - loss: 0.2307 - accuracy: 0.9513 - 705ms/epoch - 41ms/step
Epoch 7/30
17/17 - 1s - loss: 0.2511 - accuracy: 0.9438 - 796ms/epoch - 47ms/step
Epoch 8/30
17/17 - 1s - loss: 0.2555 - accuracy: 0.9457 - 920ms/epoch - 54ms/step
Epoch 9/30
17/17 - 1s - loss: 0.2254 - accuracy: 0.9457 - 685ms/epoch - 40ms/step
Epoch 10/30
17/17 - 1s - loss: 0.2289 - accuracy: 0.9457 - 695ms/epoch - 41ms/step
Epoch 11/30
17/17 - 1s - loss: 0.2282 - accuracy: 0.9438 - 693ms/epoch - 41ms/step
Epoch 12/30
17/17 - 1s - loss: 0.2395 - accuracy: 0.9382 - 700ms/epoch - 41ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
25/25 - 3s - loss: 0.6936 - accuracy: 0.5025 - 3s/epoch - 132ms/step
Epoch 2/30
25/25 - 1s - loss: 0.6913 - accuracy: 0.5275 - 754ms/epoch - 30ms/step
Epoch 3/30
25/25 - 1s - loss: 0.4858 - accuracy: 0.7900 - 768ms/epoch - 31ms/step
Epoch 4/30
25/25 - 1s - loss: 0.4127 - accuracy: 0.8900 - 771ms/epoch - 31ms/step
Epoch 5/30
25/25 - 1s - loss: 0.3431 - accuracy: 0.9062 - 732ms/epoch - 29ms/step
Epoch 6/30
25/25 - 1s - loss: 0.2426 - accuracy: 0.9350 - 1s/epoch - 42ms/step
Epoch 7/30
25/25 - 1s - loss: 0.1493 - accuracy: 0.9750 - 878ms/epoch - 35ms/step
Epoch 8/30
25/25 - 1s - loss: 0.0939 - accuracy: 0.9775 - 771ms/epoch - 31ms/step
Epoch 9/30
25/25 - 1s - loss: 0.0810 - accuracy: 0.9837 - 748ms/epoch - 30ms/step
Epoch 10/30
25/25 - 1s - loss: 0.0632 - accuracy: 0.9875 - 730ms/epoch - 29ms/step
Epoch 11/30
25/25 - 1s - loss: 0.0653 - accuracy: 0.9812 - 759ms/epoch - 30ms/step
Epoch 12/30
25/25 - 1s - loss: 0.0372 - accuracy: 0.9925 - 758ms/epoch - 30ms/step
Epoch 13/30
25/25 

In [126]:
results_df_lstm = pd.DataFrame(grid_result_lstm.cv_results_)
results_df_lstm[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.926093,10
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.922583,11
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.963706,6
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.970028,5
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.933692,9
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.978753,2
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.821459,15
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.976270,3
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.900059,12
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.942506,8


In [127]:
print("Best Parameters (LSTM): ", grid_result_lstm.best_params_)
print("Best Accuracy (LSTM): ", grid_result_lstm.best_score_)

Best Parameters (LSTM):  {'learning_rate': 0.0001, 'optimizer': 'SGD'}
Best Accuracy (LSTM):  0.983756206996893


In [128]:
best_lstm_model = grid_result_lstm.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_lstm_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)


16/16 - 1s - 701ms/epoch - 44ms/step
Test Accuracy: 0.978


In [129]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='spam')
recall_spam = recall_score(y_test, y_pred, pos_label='spam')
f1_spam = f1_score(y_test, y_pred, pos_label='spam')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not_spam')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not_spam')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not_spam')

print("Precision (spam):", precision_spam)
print("Recall (spam):", recall_spam)
print("F1-score (spam):", f1_spam)

print("Precision (non-spam):", precision_non_spam)
print("Recall (non-spam):", recall_non_spam)
print("F1-score (non-spam):", f1_non_spam)


Precision (spam): 0.9838056680161943
Recall (spam): 0.972
F1-score (spam): 0.977867203219316
Precision (non-spam): 0.9723320158102767
Recall (non-spam): 0.984
F1-score (non-spam): 0.978131212723658
